In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

from datetime import datetime
from datetime import date

In [29]:
df_model = pd.read_csv("../inter_data/processed_catboost.csv")


In [30]:
df_model = pd.read_csv("../inter_data/processed_catboost.csv")
df_model = df_model[list(col for col in df_model.columns if "Unnam" not in col)]

## Loading the model

In [31]:
from catboost import CatBoostClassifier
cat_clf = CatBoostClassifier()      # parameters not required.
cat_clf.load_model('../models/catboost_classifier')

## karvy clients not traded post ETF

In [32]:
## Karvy clients and not traded
df = df_model[(df_model["IsKarvy"] == 1) & (df_model["target"] == 0)]

In [33]:
df

,client_code,KRA YN,IsKarvy,cm_sex,IncomePA,IsDormant,E2Channel,age,months_since_last_login,months_since_last_futures_trade,months_since_last_options_trade,months_since_last_intraday_trade,months_since_last_delivery_trade,THV,ALB,Z20,total_logins_one_year,total_trades_one_year,target
157,62720305,N,1.0,M,5_10L,N,Franchisee,49.0,2.094344,132.0,132.0,132.0,3.441394,0.0,-26.20,0.0,95.0,2.0,0.0
203,IJACO324,N,1.0,M,5_10L,N,Premia,55.0,0.188761,132.0,132.0,132.0,3.474249,12982.0,0.74,0.0,343.0,2.0,0.0
240,K8260291,N,1.0,M,5_10L,N,CAT,35.0,3.112845,132.0,132.0,132.0,3.441394,0.0,-26.19,0.0,73.0,2.0,0.0
465,KL259376,N,1.0,M,5_10L,N,CAT,77.0,132.000000,132.0,132.0,132.0,3.441394,0.0,-28.19,0.0,16.0,2.0,0.0
536,ASHVICHA,N,1.0,F,<=1L,N,Franchisee,35.0,3.342830,132.0,132.0,132.0,3.474249,30.0,0.55,0.0,30.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320610,RAMNEEKU,N,1.0,M,5_10L,N,CAT,69.0,0.155906,132.0,132.0,132.0,0.418745,82316.0,61324.77,0.0,26.0,6.0,0.0
320613,VANDSHA6,N,1.0,F,5_10L,N,Franchisee,50.0,0.155906,132.0,132.0,132.0,0.155906,40.0,115.40,0.0,20.0,2.0,0.0
320615,BAJRANDE,N,1.0,M,5_10L,N,Franchisee,26.0,0.155906,132.0,132.0,132.0,0.188761,809.0,303.99,0.0,49.0,5.0,0.0
320616,JITENMAD,N,1.0,M,5_10L,N,Franchisee,50.0,0.155906,132.0,132.0,132.0,0.221616,30.0,25.69,0.0,18.0,2.0,0.0


In [34]:
X = df.drop("target", axis=1)
y = df["target"]

# removing client_code
X = X.drop("client_code", axis = 1)

X["IsKarvy"] = X["IsKarvy"].astype(int)
X["Z20"] = X["Z20"].astype(int)


In [44]:
y_pred = cat_clf.predict(X)

In [48]:
results = pd.DataFrame({'Converted':y, 'Converted_prob':y_pred, "prospect_ID": y.index})


In [55]:
df_shortlist = df.loc[list(results[results["Converted_prob"] == 1]["prospect_ID"])]
bins = [0, 35, 50, 100]
df_shortlist['age_binned'] = pd.cut(df_shortlist['age'], bins)

In [58]:
pd.DataFrame(df_shortlist["age_binned"].value_counts())

,age_binned
"(35, 50]",315
"(50, 100]",303
"(0, 35]",239


In [62]:
df_model[(df_model["total_trades_one_year"] < 10) & (df_model["target"] == 1)]

,client_code,KRA YN,IsKarvy,cm_sex,IncomePA,IsDormant,E2Channel,age,months_since_last_login,months_since_last_futures_trade,months_since_last_options_trade,months_since_last_intraday_trade,months_since_last_delivery_trade,THV,ALB,Z20,total_logins_one_year,total_trades_one_year,target
4,SANJU037,N,0.0,M,1_5L,N,GL Investments,32.0,0.287325,132.0,3.375684,3.441394,3.375684,0.0,-3.88,0.0,207.0,7.0,1.0
5,VADILPAT,Y,0.0,M,1_5L,N,Franchisee,58.0,3.539959,132.0,132.000000,132.000000,3.441394,0.0,-26.15,1.0,10.0,2.0,1.0
6,SMPSINGH,N,0.0,M,1_5L,Y,CAT,23.0,0.320180,132.0,132.000000,132.000000,3.441394,0.0,-27.51,0.0,105.0,2.0,1.0
14,FI970PRB,Y,0.0,M,1_5L,N,Franchisee,44.0,3.474249,132.0,132.000000,132.000000,0.813004,0.0,5168.50,1.0,151.0,4.0,1.0
17,12627074,Y,0.0,F,<=1L,N,CAT,35.0,3.375684,132.0,132.000000,132.000000,2.751442,0.0,-24.24,1.0,50.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315129,69396873,N,0.0,M,<=1L,N,CAT,45.0,0.155906,132.0,132.000000,132.000000,0.155906,26.0,24.97,1.0,3.0,2.0,1.0
315149,74514159,Y,0.0,F,10_25L,N,CAT,34.0,0.155906,132.0,132.000000,132.000000,0.155906,38825.0,49413.29,1.0,44.0,6.0,1.0
315190,84882940,Y,0.0,M,<=1L,N,CAT,37.0,0.155906,132.0,132.000000,132.000000,0.155906,3284.0,6716.45,1.0,20.0,2.0,1.0
315216,91474176,Y,0.0,M,<=1L,N,CAT,24.0,0.155906,132.0,132.000000,132.000000,0.155906,13.0,3.26,1.0,2.0,2.0,1.0
